In [ ]:
import os
from langchain.agents import initialize_agent, Tool
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.prompts import StringPromptTemplate
from langchain import OpenAI, SerpAPIWrapper, LLMChain
from typing import List, Union
from langchain.schema import AgentAction, AgentFinish
import re
os.environ["OPENAI_API_KEY"] = "sk-"

@tool("save_goals",return_direct=True,)
def save_goals(query: str) -> str:
    assessmentdb = Chroma(collection_name=f'x{userid}assessment',persist_directory=f'{vectorstore_folder}/assessment', embedding_function=embeddings)
    progress_file = f'progress/{userid}/progress.txt'
    questions = read_file(progress_file)
    if len(questions) == 0:
        response,check = Assessment_bot(query,vectordb,formatted_timestamp,userid)
        print(response)
    else:
        question = questions[0]
        print(question)
        remove_first_line(progress_file)
        Chroma.add_texts(assessmentdb,texts=[f"\n===============================\nConversation-Time-Stamp:{formatted_timestamp}\nDiya:{question}\n"])
    if not check:
        Chroma.add_texts(assessmentdb,texts=[f"Human: {query}\n\n"])

    assessmentdb.persist()
    assessmentdb = None

tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to answer questions about current events"
    )
]

# Set up the base template
template = """Answer the following questions as best you can, 
Your name is Diya and you are a wellness coach. You are 25 year old female and you are kind and open hearted and chats in friendly tone that allows users to confide in you. You are an expert in Cognitive Behavioral Therapy (CBT) principles and understand all the stages involved in CBT to help the uses.
You should always use the following rules while having the conversations:
- your questions and conversations should not be very long and be around 50 words.
- you should always ask one question at a time.
- you are encouraged to listen so that the users can confide in you.

You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

REMEMBER TO ALWAYS FOLLOW THE FORMAT
IF YOU THINK CONVERSATION IS INTRODUCTIONS THEN
Action: [Reply directly to the user]
Final Answer: [final directly generated answer]

IF YOU THINK YOU DONT HAVE THE AVAILABLE INFORMATION YOU SHOULD USE THE TOOLS FOR THAT STAGE AND RETURN THEIR ANSWERS:
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

ALWAYS FOLLOW THIS PATTERN USUALLY THOUGHT TELLS WHAT ACTION IS THE BOT TO GIVE. ACTION CAN BE THE ANSWER.


Question: {input}
{agent_scratchpad}"""



# Set up a prompt template
class CustomPromptTemplate(StringPromptTemplate):
    # The template to use
    template: str
    # The list of tools available
    tools: List[Tool]
    
    def format(self, **kwargs) -> str:
        # Get the intermediate steps (AgentAction, Observation tuples)
        # Format them in a particular way
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts
        # Create a tools variable from the list of tools provided
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
        # Create a list of tool names for the tools provided
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
        return self.template.format(**kwargs)
    
    
    
prompt = CustomPromptTemplate(
    template=template,
    tools=tools,
    # This omits the `agent_scratchpad`, `tools`, and `tool_names` variables because those are generated dynamically
    # This includes the `intermediate_steps` variable because that is needed
    input_variables=["input", "intermediate_steps"]
)



class CustomOutputParser(AgentOutputParser):
    
    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        # Check if agent should finish
        if "Final Answer:" in llm_output:
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        # Parse out the action and action input
        regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            raise ValueError(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        # Return the action and action input
        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)

output_parser = CustomOutputParser()
llm = ChatOpenAI(temperature=0.3)
llm_chain = LLMChain(llm=llm, prompt=prompt)
tool_names = [tool.name for tool in tools]
agent = LLMSingleActionAgent(
    llm_chain=llm_chain, 
    output_parser=output_parser,
    stop=["\nObservation:"], 
    allowed_tools=tool_names
)
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=False)